<a href="https://colab.research.google.com/github/virginiakm1988/VGC-adapter/blob/main/Adapters_are_all_you_need.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install package && setup


In [ ]:
! pip install transformers datasets
! pip install loralib

In [ ]:
!nvidia-smi

Wed Apr  5 11:20:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    46W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Define custom adapter modules
Here we implemented `Houlsby`, `ConvAdapters`, `AdapterBias`, and `LoRA`.
1. Houlsby Adapter ([Parameter-Efficient Transfer Learning for NLP](https://http://proceedings.mlr.press/v97/houlsby19a.html))
2. ConvAdapter ([CHAPTER: Exploiting Convolutional Neural Network Adapters for Self-supervised Speech Models](https://arxiv.org/abs/2212.01282))
3. AdapterBias ([AdapterBias: Parameter-efficient Token-dependent Representation Shift for Adapters in NLP Tasks](https://arxiv.org/abs/2205.00305))

4. LoRA ([LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685))

5. BitFit ([BitFit: Simple Parameter-efficient Fine-tuning for Transformer-based Masked Language-models](https://arxiv.org/abs/2106.10199)
```
mark_only_adapter_as_trainable(model_bert,bias="all")
```



In [69]:
from torch import nn
  
## Houlsby adapter
class Houlsby_Adapter(nn.Module):
    def __init__(
            self,
            input_size,
            bottleneck = 32
        ):
        super().__init__()

        self.houlsby_adapter = nn.Sequential(
          nn.Linear(input_size, bottleneck),
          nn.GELU(),
          nn.Linear(bottleneck, input_size),
      )
    def forward(self, x):
        return self.houlsby_adapter(x)

## conv adapter
class Conv_Adapter(nn.Module):
    def __init__(
            self,
            input_size,
            compress_rate = 8,
            k = 1, 
            stride = 1,
            dropout = 0.8
        ):
        super().__init__()
        def depthwise_conv(n_in, n_out, compress_rate, k, stride):
          conv = nn.Conv1d(n_in, n_out//compress_rate, k, stride = stride)
          nn.init.kaiming_normal_(conv.weight)
          return conv
        def pointwise_conv(n_in, n_out, compress_rate, k, stride):
          conv = nn.Conv1d(n_out//compress_rate,n_out, 1)
          nn.init.kaiming_normal_(conv.weight)
          return conv
        self.conv_adapter = nn.Sequential(
        depthwise_conv(input_size, input_size, compress_rate,k ,stride),
        pointwise_conv(input_size, input_size, compress_rate,k ,stride),
        nn.Dropout(p=dropout),
        nn.GELU()
      )
    def forward(self, x):
        return self.conv_adapter(x)

## adapterBias
class AdapterBias(nn.Module):
    def __init__(
            self,
            input_size,
            dropout = 0.8
        ):
        super().__init__()
        self.adapter_vector = nn.Parameter(torch.ones((input_size), requires_grad=True))
        self.adapter_alpha = nn.Linear(input_size, 1)
        
    def forward(self, x):
        return self.adapter_vector  * self.adapter_alpha(x)
##lora
class LoRA(nn.Module):
    def __init__(
            self,
            input_size,
            dropout = 0.8,
            r = 16
        ):
        super().__init__()
        self.lora_adapter = lora.Linear(input_size, input_size, r)
        
    def forward(self, x):
        return self.lora_adapter(x)

## Utils

In [98]:
from typing import Dict
def mark_only_adapter_as_trainable(model: nn.Module, bias: str = 'none') -> None:
    for n, p in model.named_parameters():
        
        if 'adapter' not in n:
          p.requires_grad = False
        else:
          p.requires_grad = True
    if bias == "none":
      return
    elif bias == 'all':
        for n, p in model.named_parameters():
            if 'bias' in n:
                p.requires_grad = True
    else:
        raise NotImplementedError


def adapter_state_dict(model: nn.Module, bias: str = 'none') -> Dict[str, torch.Tensor]:
    my_state_dict = model.state_dict()
    if bias == 'none':
        return {k: my_state_dict[k] for k in my_state_dict if 'adapter' in k}
    elif bias == 'all':
        return {k: my_state_dict[k] for k in my_state_dict if 'adapter_' in k or 'bias' in k}
    else:
        raise NotImplementedError

# Adding adapter to a hugginface model

In [99]:
from transformers import AutoModelForSequenceClassification

from torch import nn
import torch

BertLayerNorm = torch.nn.LayerNorm

##vanilla houlsby residual adapter, custom layers
class adapted_bert_output(nn.Module):
  def __init__(self, BertOutput, config):
    super().__init__()
    self.config = config
    self.dense = nn.Linear(config.intermediate_size, config.hidden_size)
    self.LayerNorm = BertLayerNorm(config.hidden_size, eps=config.layer_norm_eps)
    self.dropout = nn.Dropout(config.hidden_dropout_prob)

    if config.adapter == "houlsby":
      self.adapter = Houlsby_Adapter(config.hidden_size)
    elif config.adapter == "conv_adapter":
      self.adapter = Conv_Adapter(config.max_position_embeddings)
    elif self.adapter == "AdapterBias":
      self.adapter = AdapterBias(config.hidden_size)
    elif self.adapter == "lora":
      self.adapter = LoRA(config.hidden_size)
    else:
      raise NotImplementedError

  def forward(self,  hidden_states, input_tensor):

    hidden_states = self.dense(hidden_states)
    if self.config.adapter != None:
      adapter_output = self.adapter(hidden_states)
      hidden_states = self.dropout(hidden_states) + adapter_output
    else:
      hidden_states = self.dropout(hidden_states)
    hidden_states = self.LayerNorm(hidden_states + input_tensor)
  
    return hidden_states


model_bert = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
model_bert.config.adapter = "houlsby"
#add adapter module in a bert model
for idx, layer in enumerate(model_bert.bert.encoder.layer):
  model_bert.bert.encoder.layer[idx].output = adapted_bert_output(model_bert.bert.encoder.layer[idx].output, model_bert.config)

#freeze parameters
mark_only_adapter_as_trainable(model_bert)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

# Loading datasets

In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset("imdb")
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]


# Start training

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir = "test-trainer",per_device_train_batch_size = 4)
trainer = Trainer(
    model=model_bert, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset
)
trainer.train() 

Step,Training Loss
500,0.706200


TrainOutput(global_step=750, training_loss=0.7052169596354166, metrics={'train_runtime': 93.2405, 'train_samples_per_second': 32.175, 'train_steps_per_second': 8.044, 'total_flos': 792051075072000.0, 'train_loss': 0.7052169596354166, 'epoch': 3.0})

# Saving adapter checkpoints


In [95]:
checkpoint_path = "./result"
torch.save(adapter_state_dict(model_bert), checkpoint_path)

# Adding LoRA in selfattention

In [ ]:
import loralib as lora
import math
class AdaptedBertSelfAttention(nn.Module):
  def __init__(self, BertSelfAttention, config):
    super().__init__()
    self.num_attention_heads = config.num_attention_heads
    self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
    self.all_head_size = self.num_attention_heads * self.attention_head_size
    self.query = lora.Linear(config.hidden_size, self.all_head_size, config.lora_r, lora_alpha=config.lora_alpha)
    self.key = nn.Linear(config.hidden_size, self.all_head_size)
    self.value = lora.Linear(config.hidden_size, self.all_head_size, config.lora_r, lora_alpha=config.lora_alpha)

    self.dropout = nn.Dropout(config.attention_probs_dropout_prob)
    self.position_embedding_type = getattr(config, "position_embedding_type", "absolute")
    if self.position_embedding_type == "relative_key" or self.position_embedding_type == "relative_key_query":
            self.max_position_embeddings = config.max_position_embeddings
            self.distance_embedding = nn.Embedding(2 * config.max_position_embeddings - 1, self.attention_head_size)

    self.is_decoder = config.is_decoder

  def transpose_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)

  def forward(
        self,
        hidden_states,
        attention_mask=None,
        head_mask=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        past_key_value=None,
        output_attentions=False,
    ):
        mixed_query_layer = self.query(hidden_states)

        # If this is instantiated as a cross-attention module, the keys
        # and values come from an encoder; the attention mask needs to be
        # such that the encoder's padding tokens are not attended to.
        is_cross_attention = encoder_hidden_states is not None

        if is_cross_attention and past_key_value is not None:
            # reuse k,v, cross_attentions
            key_layer = past_key_value[0]
            value_layer = past_key_value[1]
            attention_mask = encoder_attention_mask
        elif is_cross_attention:
            key_layer = self.transpose_for_scores(self.key(encoder_hidden_states))
            value_layer = self.transpose_for_scores(self.value(encoder_hidden_states))
            attention_mask = encoder_attention_mask
        elif past_key_value is not None:
            key_layer = self.transpose_for_scores(self.key(hidden_states))
            value_layer = self.transpose_for_scores(self.value(hidden_states))
            key_layer = torch.cat([past_key_value[0], key_layer], dim=2)
            value_layer = torch.cat([past_key_value[1], value_layer], dim=2)
        else:
            key_layer = self.transpose_for_scores(self.key(hidden_states))
            value_layer = self.transpose_for_scores(self.value(hidden_states))

        query_layer = self.transpose_for_scores(mixed_query_layer)

        if self.is_decoder:
            # if cross_attention save Tuple(torch.Tensor, torch.Tensor) of all cross attention key/value_states.
            # Further calls to cross_attention layer can then reuse all cross-attention
            # key/value_states (first "if" case)
            # if uni-directional self-attention (decoder) save Tuple(torch.Tensor, torch.Tensor) of
            # all previous decoder key/value_states. Further calls to uni-directional self-attention
            # can concat previous decoder key/value_states to current projected key/value_states (third "elif" case)
            # if encoder bi-directional self-attention `past_key_value` is always `None`
            past_key_value = (key_layer, value_layer)

        # Take the dot product between "query" and "key" to get the raw attention scores.
        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))

        if self.position_embedding_type == "relative_key" or self.position_embedding_type == "relative_key_query":
            seq_length = hidden_states.size()[1]
            position_ids_l = torch.arange(seq_length, dtype=torch.long, device=hidden_states.device).view(-1, 1)
            position_ids_r = torch.arange(seq_length, dtype=torch.long, device=hidden_states.device).view(1, -1)
            distance = position_ids_l - position_ids_r
            positional_embedding = self.distance_embedding(distance + self.max_position_embeddings - 1)
            positional_embedding = positional_embedding.to(dtype=query_layer.dtype)  # fp16 compatibility

            if self.position_embedding_type == "relative_key":
                relative_position_scores = torch.einsum("bhld,lrd->bhlr", query_layer, positional_embedding)
                attention_scores = attention_scores + relative_position_scores
            elif self.position_embedding_type == "relative_key_query":
                relative_position_scores_query = torch.einsum("bhld,lrd->bhlr", query_layer, positional_embedding)
                relative_position_scores_key = torch.einsum("bhrd,lrd->bhlr", key_layer, positional_embedding)
                attention_scores = attention_scores + relative_position_scores_query + relative_position_scores_key

        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        if attention_mask is not None:
            # Apply the attention mask is (precomputed for all layers in RobertaModel forward() function)
            attention_scores = attention_scores + attention_mask

        # Normalize the attention scores to probabilities.
        attention_probs = nn.Softmax(dim=-1)(attention_scores)

        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        attention_probs = self.dropout(attention_probs)

        # Mask heads if we want to
        if head_mask is not None:
            attention_probs = attention_probs * head_mask

        context_layer = torch.matmul(attention_probs, value_layer)

        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)

        outputs = (context_layer, attention_probs) if output_attentions else (context_layer,)

        if self.is_decoder:
            outputs = outputs + (past_key_value,)
        return outputs

In [ ]:
model_bert = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

#add lora adapter module in a bert model by replacing the selfattention function
model_bert.config.lora_r = 8
model_bert.config.lora_alpha = 8
for idx, layer in enumerate(model_bert.bert.encoder.layer):
  model_bert.bert.encoder.layer[idx].attention.self = AdaptedBertSelfAttention(model_bert.bert.encoder.layer[idx].attention.self, model_bert.config)
for name, param in model_bert.named_parameters():
  if "lora" in name:
    param.require_grad = True
  else:
    param.require_grad = False

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments("test_trainer")
trainer = Trainer(
    model=model_bert, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset
)

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=375, training_loss=0.6475082600911458, metrics={'train_runtime': 285.2805, 'train_samples_per_second': 10.516, 'train_steps_per_second': 1.314, 'total_flos': 1056031524864000.0, 'train_loss': 0.6475082600911458, 'epoch': 3.0})